### introduction

矩阵乘法经常出现在实际是代数运算中，被各种计算库如eigen，mkl，openblas集成，也经常用于基准测试中。在尺度较小时，数据量近似等于计算量，如果尺度翻倍，计算量变为原先的8倍，但数据量仅为原先的4倍，存在随尺度增大的数据局域性。由于其独特的计算量和数据量增长关系，一个优秀的矩阵相乘算法必须充分利用各级存储资源。

### what should you do
你需要从文件conf.data中读取两个矩阵m1，m2并计算乘积m3=m1\*m2，然后将结果写入文件out.data中。

输入文件定义如下  
三个64位整数 n1,n2,n3  
第一个矩阵m1的数据，矩阵有n1行n2列，64位浮点，行优先。  
第二个矩阵m2的数据，矩阵有n2行n3列，64位浮点，行优先。  

将矩阵m3直接写入输出文件中，64位浮点，行优先。

提供样例程序以供参考

### example
~~~
#include <iostream>
#include <chrono>

void mul(double* a, double* b, double* c, uint64_t n1, uint64_t n2, uint64_t n3) {
	for (int i = 0; i < n1; i++) {
		for (int j = 0; j < n2; j++) {
			for (int k = 0; k < n3; k++) {
				c[i * n3 + k] += a[i * n2 + j] * b[j * n3 + k];
			}
		}
	}
}

int main() {
	uint64_t n1, n2, n3;
	FILE* fi;

	fi = fopen("conf.data", "rb");
	fread(&n1, 1, 8, fi);
	fread(&n2, 1, 8, fi);
	fread(&n3, 1, 8, fi);

	double* a = (double*)malloc(n1 * n2 * 8);
	double* b = (double*)malloc(n2 * n3 * 8);
	double* c = (double*)malloc(n1 * n3 * 8);

	fread(a, 1, n1 * n2 * 8, fi);
	fread(b, 1, n2 * n3 * 8, fi);
	fclose(fi);

	for (uint64_t i = 0; i < n1; i++) {
		for (uint64_t k = 0; k < n3; k++) {
			c[i * n3 + k] = 0;
		}
	}

	auto t1 = std::chrono::steady_clock::now();
	mul(a, b, c, n1, n2, n3);
	auto t2 = std::chrono::steady_clock::now();
	int d1 = std::chrono::duration_cast<std::chrono::milliseconds>(t2 - t1).count();
	printf("%d\n", d1);


	fi = fopen("out.data", "wb");
	fwrite(c, 1, n1 * n3 * 8, fi);
	fclose(fi);

	return 0;
}
~~~

### benchmark details
提交单个cpp文件，提交的代码会以最简单的方式编译并执行，编译选项为-O3 -fopenmp -march=native，通常编译器会尝试自动avx512向量化，也可以使用avx512内置函数。如果需要链接特定系统或编译器自带库，请联系工作人员并说明理由，编译条件的更改会对所有选手可见。程序运行在AMD机器中的8个核心上。  
各测试点如下所示，在限制时间内得到正确结果可以得到基本分，在满分时间内得到正确结果可以得到满分，分数随时间的对数线性变化。除第一个测试点外，时间限制小于样例程序用时，意味着提交样例程序无法获得分数。

|序号|n1,n2,n3|样例程序用时|时间限制|满分时间|基本分值|满分分值|
|----|----|----|----|----|----|----|
|1|1024，1024，1024|0.15s|1s|0.1s|20|20|
|2|8192，8192，8192|2min|40s|4s|10|60|
|3|8192，16384，8192|4min|80s|7.8s|10|60|
|4|16384，16384，8192|8min|160s|15s|10|60|